In [1]:
1+1

2

In [5]:
import os
import pandas as pd
import geopandas as gpd
import rioxarray as rxr
import matplotlib.pyplot as plt
from geocube.api.core import make_geocube
from xrspatial import zonal_stats

In [33]:
# gdf = gpd.read_file("inputs/sabre_bomgrid_within6km_of_plantation.geojson")
gdf = gpd.read_file('../data/inputs/h3.gpkg', layer='h3_elliott_river')
gdf = gdf.iloc[:256]

display(gdf.head())

,GRID_ID,level,within0km,within10km,MERGE_SRC,geometry
0,85be8b63fffffff,5,1,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,"MULTIPOLYGON (((16956166.438 -2875341.283, 169..."
1,85be8b73fffffff,5,1,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,"MULTIPOLYGON (((16970052.456 -2890108.298, 169..."
2,85be8b77fffffff,5,1,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,"MULTIPOLYGON (((16950083.811 -2894649.742, 169..."
3,86be8b607ffffff,6,1,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,"MULTIPOLYGON (((16948489.353 -2874524.713, 169..."
4,86be8b627ffffff,6,1,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,"MULTIPOLYGON (((16951580.613 -2881500.752, 169..."


In [8]:
import sys
sys.path.append('..')
import utilities

In [34]:
# resource = utilities.load_resource("../resources/pc-sentinel-2-l2a.yaml")
resource = utilities.load_resource("../resources/dea-ga_s2bm_ard_3.yaml")

url = resource["url"]
sensor_name = resource["name"]
bands = resource["bands"]
print(f"Bands to be collected: {bands}")

bounds = gdf.to_crs("EPSG: 4326").total_bounds.tolist()

time_range = "2025-12-01/2025-12-31"
stac_data = utilities.get_data_from_stac(
    url=url,
    bounds=bounds,
    sensor_name=sensor_name,
    sensor_bands=bands,
    time_range=time_range)

print(f"Raw data size {utilities.calculate_data_size_in_gb(data):.2g} GB")

Bands to be collected: ['nbart_blue', 'nbart_green', 'nbart_red', 'nbart_nir_1', 'nbart_swir_2', 'nbart_swir_3', 'oa_fmask']
Raw data size 2.3 GB


In [35]:
gdf['zone_id'] = gdf.index

In [36]:
geocube = make_geocube(gdf, like=stac_data)

In [ ]:
import xarray as xr
import numpy as np
import dask.array as da
from dask.distributed import Client

# Optional: start a local Dask cluster for parallel processing
client = Client()

# --- Assumptions ---
# `geocube`  : xr.Dataset with a rasterized zone ID variable (e.g., "zone_id")
#              dims: (y, x), chunked to match stac_data
# `stac_data`: xr.Dataset with Sentinel-2 bands
#              dims: (time, y, x), lazily loaded via dask

# 1. Align spatial coordinates (ensure both arrays share the same grid)
geocube_aligned = geocube.rio.reproject_match(stac_data)

# 2. Extract the zone ID mask (integer labels per pixel)
zone_mask = geocube_aligned["zone_id"]  # shape: (y, x)

# 3. Rechunk for efficient groupby — align chunks on spatial dims
stac_data = stac_data.chunk({"time": 1, "y": 512, "x": 512})
zone_mask = zone_mask.chunk({"y": 512, "x": 512})

# 4. Compute unique zone labels upfront (required for chunked groupby)
unique_zone_labels = np.unique(zone_mask.values)
unique_zone_labels = unique_zone_labels[~np.isnan(unique_zone_labels.astype(float))]

# 5. Add the zone mask as a coordinate on the STAC data
stac_data = stac_data.assign_coords(zone=zone_mask)

# 6. Compute zonal statistics for all bands using UniqueGrouper with explicit labels
zonal_stats_all_bands = xr.Dataset({
    band: stac_data[band]
        .groupby(zone=xr.groupers.UniqueGrouper(labels=unique_zone_labels))
        .mean(dim=["y", "x"])
    for band in bands
}).compute()

/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/distributed/node.py:188: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38593 instead
  warnings.warn(
/home/ben/miniconda3/envs/forestHealth/lib/python3.14/site-packages/distributed/client.py:3374: UserWarning: Sending large graph of size 112.61 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
